# Telugu to English Translation & Scoring using SeamlessM4T

This notebook translates Telugu audio files to English using Meta's **SeamlessM4T** model and compares the results with human transcriptions using comprehensive evaluation metrics.

## Features:
- 🎯 Telugu → English speech translation using SeamlessM4T
- 📊 Multiple evaluation metrics (BLEU, ChrF++, TER, ROUGE, METEOR, BERTScore, WER, CER)
- 💾 Progress tracking with caching
- 📈 Detailed visualization and per-file analysis
- 📁 Results export to JSON and CSV

## Step 1: Install Dependencies

Installing all required packages for SeamlessM4T and evaluation metrics.

In [1]:
# Install core dependencies
!pip install --upgrade pip
!pip install torch torchaudio transformers sentencepiece
!pip install accelerate

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.11.3-cp314-cp314-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/109.3 MB 2.7 MB/s eta 0:00:41
   ---------------------------------------- 0.8/109.3 MB 1.7 MB/s eta 0:01:05
   --------------

In [2]:
# Install evaluation metrics packages
!pip install sacrebleu jiwer rouge-score nltk bert-score pandas matplotlib seaborn

  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Using cached jiwer-4.0.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached portalocker-3.2.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached rapidfuzz-3.14.3-cp314-cp314-win_amd64.whl.metadata (12 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached pywin32-311-cp314-c

In [3]:
# Download NLTK data for METEOR
import nltk
try:
    nltk.download('wordnet', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    print("✅ NLTK data downloaded successfully")
except Exception as e:
    print(f"⚠️ NLTK download warning: {e}")

✅ NLTK data downloaded successfully


## Step 2: Setup and Configuration

Configure paths, device, and check system information.

In [1]:
import os
import json
import pickle
from pathlib import Path
import datetime
import warnings
warnings.filterwarnings('ignore')

import torch
import torchaudio
from transformers import AutoProcessor, SeamlessM4Tv2Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("=" * 60)
print("SYSTEM INFORMATION")
print("=" * 60)

# Detect device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ CUDA available: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("✅ Apple Silicon (MPS) detected")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU (this will be slow)")

print(f"📍 Device: {device}")
print(f"🐍 PyTorch version: {torch.__version__}")
print("=" * 60)

SYSTEM INFORMATION
⚠️ Using CPU (this will be slow)
📍 Device: cpu
🐍 PyTorch version: 2.10.0.dev20251106+cpu


In [6]:
# Configure file paths
cwd = Path.cwd()

# Updated paths for your data - REFERENCE.txt is in parent directory
AUDIO_FOLDER = cwd / "TG"  # Folder containing Telugu audio files
REFERENCE_FILE = cwd.parent / "REFERENCE.txt"  # Human transcriptions file (in parent dir)
OUTPUT_FILE = cwd / "seamless_translations.txt"  # Model translations output
CACHE_DIR = cwd / ".seamless_cache"
RESULTS_DIR = cwd / "results"

# Create directories
CACHE_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Cache files
PROGRESS_FILE = CACHE_DIR / "translation_progress.json"
RESULTS_CACHE = CACHE_DIR / "translation_cache.pkl"

print("📂 Configuration:")
print(f"   Audio folder: {AUDIO_FOLDER}")
print(f"   Reference file: {REFERENCE_FILE}")
print(f"   Output file: {OUTPUT_FILE}")
print(f"   Cache directory: {CACHE_DIR}")
print(f"   Results directory: {RESULTS_DIR}")

# Check if audio folder exists
if not AUDIO_FOLDER.exists():
    print(f"\n⚠️ Audio folder not found at {AUDIO_FOLDER}")
    print("Please update the AUDIO_FOLDER path in the cell above to point to your Telugu audio files.")
    user_path = input("Enter the path to your audio folder (or press Enter to skip): ").strip()
    if user_path:
        AUDIO_FOLDER = Path(user_path)
        if AUDIO_FOLDER.exists():
            print(f"✅ Updated to: {AUDIO_FOLDER}")
        else:
            print(f"❌ Path still not found: {AUDIO_FOLDER}")

📂 Configuration:
   Audio folder: c:\Users\nani\Desktop\sonar\repo\TG
   Reference file: c:\Users\nani\Desktop\sonar\REFERENCE.txt
   Output file: c:\Users\nani\Desktop\sonar\repo\seamless_translations.txt
   Cache directory: c:\Users\nani\Desktop\sonar\repo\.seamless_cache
   Results directory: c:\Users\nani\Desktop\sonar\repo\results


## Step 3: Load SeamlessM4T Model

Loading Meta's SeamlessM4T v2 model for Telugu → English translation.

In [3]:
print("=" * 60)
print("LOADING SEAMLESSM4T MODEL")
print("=" * 60)
print("This may take a few minutes on first run (model is ~10GB)")
print("Subsequent runs will use cached model...")

try:
    # Load processor and model
    processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
    model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")
    
    # Move model to device
    model = model.to(device)
    model.eval()  # Set to evaluation mode
    
    print(f"\n✅ SeamlessM4T loaded successfully!")
    print(f"   Model on device: {next(model.parameters()).device}")
    print(f"   Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nTroubleshooting:")
    print("1. Ensure you have enough RAM/VRAM (model needs ~12GB)")
    print("2. Check your internet connection for first-time download")
    print("3. Try restarting the notebook kernel")
    raise

LOADING SEAMLESSM4T MODEL
This may take a few minutes on first run (model is ~10GB)
Subsequent runs will use cached model...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Fetching 2 files:


✅ SeamlessM4T loaded successfully!
   Model on device: cpu
   Model size: 2.31B parameters


## Step 4: Translation Function

Define the function to translate Telugu audio to English text.

In [27]:
import soundfile as sf
import numpy as np

def translate_telugu_to_english(audio_path, max_retries=3):
    """
    Translate Telugu audio file to English text using SeamlessM4T.
    Fixed to work with all torchaudio versions using soundfile directly.
    
    Args:
        audio_path: Path to Telugu audio file (.wav format)
        max_retries: Number of retry attempts on failure
    
    Returns:
        Translated English text or error message
    """
    for attempt in range(max_retries):
        try:
            # Load audio using soundfile directly (bypasses torchaudio backend issues)
            audio, sample_rate = sf.read(str(audio_path), dtype='float32')
            
            # Convert to torch tensor
            audio = torch.from_numpy(audio)
            
            # Ensure correct shape (add channel dimension if needed)
            if audio.ndim == 1:
                audio = audio.unsqueeze(0)  # Add channel dimension
            elif audio.ndim == 2 and audio.shape[0] > audio.shape[1]:
                audio = audio.transpose(0, 1)  # Transpose if needed
            
            # Resample to 16kHz if needed (SeamlessM4T expects 16kHz)
            if sample_rate != 16000:
                resampler = torchaudio.transforms.Resample(sample_rate, 16000)
                audio = resampler(audio)
            
            # Convert to mono if stereo
            if audio.shape[0] > 1:
                audio = audio.mean(dim=0, keepdim=True)
            
            # Prepare audio for model
            audio = audio.squeeze().numpy()
            
            # Process audio and generate translation
            # Source language: Telugu (tel), Target language: English (eng)
            audio_inputs = processor(audios=audio, sampling_rate=16000, return_tensors="pt")
            
            # Move inputs to device
            audio_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                          for k, v in audio_inputs.items()}
            
            # Generate translation
            with torch.no_grad():
                output_tokens = model.generate(
                    **audio_inputs,
                    tgt_lang="eng",  # Target language: English
                    generate_speech=False  # We only want text output
                )
            
            # FIXED: model.generate() returns GenerateEncoderDecoderOutput object
            # We need to access .sequences attribute to get the actual token tensor
            translated_text = processor.decode(output_tokens.sequences[0], skip_special_tokens=True)
            
            return translated_text.strip()
            
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"   ⚠️ Attempt {attempt + 1} failed, retrying... ({str(e)[:50]})")
                continue
            else:
                return f"ERROR: {str(e)[:100]}"
    
    return "ERROR: Max retries exceeded"

# Test the function with a sample (optional)
print("✅ Translation function FIXED!")
print("   Using soundfile.read() directly (bypasses torchaudio)")
print("   CRITICAL FIX: Using output_tokens.sequences[0] instead of output_tokens[0]")
print("   Language pair: Telugu (tel) → English (eng)")


✅ Translation function FIXED!
   Using soundfile.read() directly (bypasses torchaudio)
   CRITICAL FIX: Using output_tokens.sequences[0] instead of output_tokens[0]
   Language pair: Telugu (tel) → English (eng)


## Step 5: Process Audio Files

Translate all Telugu audio files to English with progress tracking.

In [29]:
print("=" * 60)
print("PROCESSING AUDIO FILES")
print("=" * 60)

# Load previous progress if available
try:
    with open(PROGRESS_FILE, 'r') as f:
        progress_data = json.load(f)
    processed_files = set(progress_data.get('processed_files', []))
    print(f"📂 Found previous progress: {len(processed_files)} files already processed")
except (FileNotFoundError, json.JSONDecodeError):
    processed_files = set()
    print("📂 Starting fresh (no previous progress found)")

# Load cached results if available
try:
    with open(RESULTS_CACHE, 'rb') as f:
        cached_results = pickle.load(f)
    processed_files.update(cached_results.keys())
    print(f"💾 Loaded {len(cached_results)} cached translations")
except (FileNotFoundError, EOFError):
    cached_results = {}
    print("💾 No cache file found; starting with empty results")

# Get all audio files
if AUDIO_FOLDER.exists():
    audio_files = sorted(AUDIO_FOLDER.glob("*.wav"))
    if not audio_files:
        # Try other audio formats
        audio_files = sorted(AUDIO_FOLDER.glob("*.mp3")) + sorted(AUDIO_FOLDER.glob("*.flac"))
    
    total_files = len(audio_files)
    remaining_files = [f for f in audio_files if f.name not in processed_files]
    
    print(f"\n📊 Statistics:")
    print(f"   Total audio files: {total_files}")
    print(f"   Already processed: {total_files - len(remaining_files)}")
    print(f"   Remaining: {len(remaining_files)}")
    
    if total_files == 0:
        print(f"\n❌ No audio files found in {AUDIO_FOLDER}")
        print("   Supported formats: .wav, .mp3, .flac")
else:
    print(f"❌ Audio folder not found: {AUDIO_FOLDER}")
    print("Please update AUDIO_FOLDER path in the configuration cell above")
    audio_files = []
    remaining_files = []

PROCESSING AUDIO FILES
📂 Found previous progress: 2 files already processed
💾 Loaded 2 cached translations

📊 Statistics:
   Total audio files: 2667
   Already processed: 2
   Remaining: 2665


In [9]:
# FIX: Install soundfile and update translation function
print("Installing soundfile package...")
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "soundfile"])
print("✅ soundfile installed!")

# Update translation function to use soundfile backend
def translate_telugu_to_english(audio_path, max_retries=3):
    """
    Translate Telugu audio file to English text using SeamlessM4T.
    Fixed to use soundfile backend instead of torchcodec.
    """
    for attempt in range(max_retries):
        try:
            # Load audio file with soundfile backend (fixes TorchCodec error)
            audio, sample_rate = torchaudio.load(str(audio_path), backend="soundfile")
            
            # Resample to 16kHz if needed
            if sample_rate != 16000:
                resampler = torchaudio.transforms.Resample(sample_rate, 16000)
                audio = resampler(audio)
            
            # Convert to mono if stereo
            if audio.shape[0] > 1:
                audio = audio.mean(dim=0, keepdim=True)
            
            # Prepare audio for model
            audio = audio.squeeze().numpy()
            
            # Process and translate
            audio_inputs = processor(audios=audio, sampling_rate=16000, return_tensors="pt")
            audio_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                          for k, v in audio_inputs.items()}
            
            with torch.no_grad():
                output_tokens = model.generate(
                    **audio_inputs,
                    tgt_lang="eng",
                    generate_speech=False
                )
            
            translated_text = processor.decode(output_tokens[0].tolist(), skip_special_tokens=True)
            return translated_text.strip()
            
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"   ⚠️ Attempt {attempt + 1} failed, retrying... ({str(e)[:50]})")
                continue
            else:
                return f"ERROR: {str(e)[:100]}"
    
    return "ERROR: Max retries exceeded"

print("✅ Translation function updated with soundfile backend!")
print("   Now you can continue - the errors should be fixed.")

Installing soundfile package...
✅ soundfile installed!
✅ Translation function updated with soundfile backend!
   Now you can continue - the errors should be fixed.
✅ soundfile installed!
✅ Translation function updated with soundfile backend!
   Now you can continue - the errors should be fixed.


In [30]:
# Process remaining files
if remaining_files:
    print("\n" + "=" * 60)
    print("STARTING TRANSLATION")
    print("=" * 60)
    
    with open(OUTPUT_FILE, "a", encoding="utf-8") as out_f:
        try:
            for i, audio_path in enumerate(remaining_files, 1):
                filename = audio_path.name
                
                print(f"\n[{i}/{len(remaining_files)}] Processing: {filename}")
                print(f"   Size: {audio_path.stat().st_size / 1024:.1f} KB")
                
                # Translate audio
                translated_text = translate_telugu_to_english(audio_path)
                
                # Store result
                cached_results[filename] = translated_text
                processed_files.add(filename)
                
                # Write to output file
                out_f.write(f"{filename}\t{translated_text}\n")
                out_f.flush()
                
                # Preview translation
                if translated_text.startswith("ERROR:"):
                    print(f"   ❌ {translated_text}")
                else:
                    preview = translated_text[:100] + "..." if len(translated_text) > 100 else translated_text
                    print(f"   ✅ Translation: {preview}")
                
                # Save progress every 5 files
                if i % 5 == 0 or i == len(remaining_files):
                    print(f"\n💾 Saving progress ({i}/{len(remaining_files)})...")
                    
                    # Save progress
                    with open(PROGRESS_FILE, 'w') as pf:
                        json.dump({
                            'processed_files': list(processed_files),
                            'total_files': total_files,
                            'completed': len(processed_files),
                            'last_updated': datetime.datetime.now().isoformat()
                        }, pf, indent=2)
                    
                    # Save cache
                    with open(RESULTS_CACHE, 'wb') as rf:
                        pickle.dump(cached_results, rf)
            
            print("\n" + "=" * 60)
            print("✅ TRANSLATION COMPLETE!")
            print("=" * 60)
            print(f"Processed: {len(processed_files)}/{total_files} files")
            print(f"Results saved to: {OUTPUT_FILE}")
            
        except KeyboardInterrupt:
            print("\n" + "=" * 60)
            print("⏸️  INTERRUPTED BY USER")
            print("=" * 60)
            
            # Save progress
            with open(PROGRESS_FILE, 'w') as pf:
                json.dump({
                    'processed_files': list(processed_files),
                    'total_files': total_files,
                    'completed': len(processed_files),
                    'interrupted': True,
                    'last_updated': datetime.datetime.now().isoformat()
                }, pf, indent=2)
            
            # Save cache
            with open(RESULTS_CACHE, 'wb') as rf:
                pickle.dump(cached_results, rf)
            
            print(f"💾 Progress saved: {len(processed_files)}/{total_files} files")
            print("Run this cell again to resume translation")
            
        except Exception as e:
            print(f"\n❌ Unexpected error: {e}")
            raise
else:
    if audio_files:
        print("\n✅ All files already processed!")
        print(f"Total: {len(cached_results)} translations")
    else:
        print("\n⚠️ No audio files to process")


STARTING TRANSLATION

[1/2665] Processing: IIITH_DBC_PE_TEL_TG_1000_84915.wav
   Size: 469.4 KB
   ✅ Translation: Giva is a form of dance in Andhra Pradesh. I like to play with toys because they are very popular in...

[2/2665] Processing: IIITH_DBC_PE_TEL_TG_1000_84916.wav
   Size: 469.4 KB
   ✅ Translation: Giva is a form of dance in Andhra Pradesh. I like to play with toys because they are very popular in...

[2/2665] Processing: IIITH_DBC_PE_TEL_TG_1000_84916.wav
   Size: 469.4 KB
   ✅ Translation: The dance form of Ghi Andhra is called Ghi Kolatam. Everyone in my family loves Ghi Kolatam because ...

[3/2665] Processing: IIITH_DBC_PE_TEL_TG_1000_84917.wav
   Size: 469.4 KB
   ✅ Translation: The dance form of Ghi Andhra is called Ghi Kolatam. Everyone in my family loves Ghi Kolatam because ...

[3/2665] Processing: IIITH_DBC_PE_TEL_TG_1000_84917.wav
   Size: 469.4 KB
   ✅ Translation: If I want to talk about my favorite dish in Andhra cuisine, I like pappo, which is very good in A

In [24]:
# DEBUG: Check the actual function code in memory
import inspect
print("🔍 Checking the actual function code in kernel memory...")
print("=" * 60)
source = inspect.getsource(translate_telugu_to_english)
print(source)
print("=" * 60)

# Look for the problematic line
if ".tolist()" in source:
    print("\n❌ PROBLEM FOUND: Function still has .tolist() in it!")
    print("   The kernel is using an old version of the function.")
else:
    print("\n✅ Function code looks correct (no .tolist() found)")


🔍 Checking the actual function code in kernel memory...
def translate_telugu_to_english(audio_path, max_retries=3):
    """
    Translate Telugu audio file to English text using SeamlessM4T.
    Fixed to work with all torchaudio versions using soundfile directly.

    Args:
        audio_path: Path to Telugu audio file (.wav format)
        max_retries: Number of retry attempts on failure

    Returns:
        Translated English text or error message
    """
    for attempt in range(max_retries):
        try:
            # Load audio using soundfile directly (bypasses torchaudio backend issues)
            audio, sample_rate = sf.read(str(audio_path), dtype='float32')

            # Convert to torch tensor
            audio = torch.from_numpy(audio)

            # Ensure correct shape (add channel dimension if needed)
            if audio.ndim == 1:
                audio = audio.unsqueeze(0)  # Add channel dimension
            elif audio.ndim == 2 and audio.shape[0] > audio.shape[1]:


In [26]:
# DEBUG: Check what output_tokens actually contains
print("🔍 Checking the structure of output_tokens...")
print("=" * 60)

# Generate a test output
import soundfile as sf
test_audio, test_sr = sf.read(str(remaining_files[0]), dtype='float32')
test_audio = torch.from_numpy(test_audio)
if test_audio.ndim == 1:
    test_audio = test_audio.unsqueeze(0)
if test_sr != 16000:
    resampler = torchaudio.transforms.Resample(test_sr, 16000)
    test_audio = resampler(test_audio)
if test_audio.shape[0] > 1:
    test_audio = test_audio.mean(dim=0, keepdim=True)
test_audio = test_audio.squeeze().numpy()

audio_inputs = processor(audios=test_audio, sampling_rate=16000, return_tensors="pt")
audio_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in audio_inputs.items()}

with torch.no_grad():
    output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)

print(f"output_tokens type: {type(output_tokens)}")
print(f"output_tokens attributes: {dir(output_tokens)}")
print(f"\nLet's check the 'sequences' attribute:")
if hasattr(output_tokens, 'sequences'):
    print(f"  sequences type: {type(output_tokens.sequences)}")
    print(f"  sequences shape: {output_tokens.sequences.shape}")
    print(f"  sequences[0] type: {type(output_tokens.sequences[0])}")
    print(f"  sequences[0] shape: {output_tokens.sequences[0].shape}")
    print(f"  First few tokens: {output_tokens.sequences[0][:10]}")
    
    print("\n✅ Now let's try decoding with .sequences:")
    try:
        decoded = processor.decode(output_tokens.sequences[0], skip_special_tokens=True)
        print(f"SUCCESS! Decoded text: {decoded[:200]}")
    except Exception as e:
        print(f"FAILED: {e}")


🔍 Checking the structure of output_tokens...
output_tokens type: <class 'transformers.generation.utils.GenerateEncoderDecoderOutput'>
output_tokens attributes: ['__annotations__', '__class__', '__class_getitem__', '__contains__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__or__', '__post_init__', '__reduce__', '__reduce_ex__', '__replace__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', 'clear', 'copy', 'cross_attentions', 'decoder_attentions', 'decoder_hidden_states', 'encoder_attentions', 'encoder_hidden_states', 'fromkeys', 'get', 'items', 'keys', 'logits', 'mo

In [17]:
# FINAL FIX: Working translation function
import subprocess
import sys

# Install soundfile if needed
try:
    import soundfile
    print("✅ soundfile already installed")
except ImportError:
    print("Installing soundfile...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "soundfile"])
    import soundfile
    print("✅ soundfile installed!")

# Corrected translation function - works with all torchaudio versions
def translate_telugu_to_english(audio_path, max_retries=3):
    """Fixed translation function using soundfile directly"""
    for attempt in range(max_retries):
        try:
            import soundfile as sf
            import numpy as np
            
            # Load audio using soundfile directly (most reliable method)
            audio, sample_rate = sf.read(str(audio_path), dtype='float32')
            
            # Convert to torch tensor
            import torch
            audio = torch.from_numpy(audio)
            
            # Ensure correct shape (add channel dimension if needed)
            if audio.ndim == 1:
                audio = audio.unsqueeze(0)  # Add channel dimension
            elif audio.ndim == 2 and audio.shape[0] > audio.shape[1]:
                audio = audio.transpose(0, 1)  # Transpose if needed
            
            # Resample to 16kHz if needed
            if sample_rate != 16000:
                resampler = torchaudio.transforms.Resample(sample_rate, 16000)
                audio = resampler(audio)
            
            # Convert to mono if stereo
            if audio.shape[0] > 1:
                audio = audio.mean(dim=0, keepdim=True)
            
            # Prepare audio for model
            audio = audio.squeeze().numpy()
            
            # Process and translate
            audio_inputs = processor(audios=audio, sampling_rate=16000, return_tensors="pt")
            audio_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v 
                          for k, v in audio_inputs.items()}
            
            with torch.no_grad():
                output_tokens = model.generate(
                    **audio_inputs,
                    tgt_lang="eng",
                    generate_speech=False
                )
            
            # FIXED: Extract the tensor properly before decoding
            # output_tokens is a 2D tensor [batch_size, sequence_length]
            # We need to extract the first sequence
            translated_text = processor.decode(output_tokens[0], skip_special_tokens=True)
            return translated_text.strip()
            
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"   ⚠️ Attempt {attempt + 1} failed, retrying... ({str(e)[:50]})")
                continue
            else:
                return f"ERROR: {str(e)[:100]}"
    return "ERROR: Max retries exceeded"

print("✅ Function redefined - using soundfile.read() directly!")
print("   Fixed: processor.decode() now receives tensor instead of list")
print("\n🔄 Now STOP the running translation cell and RE-RUN it!")

✅ soundfile already installed
✅ Function redefined - using soundfile.read() directly!
   Fixed: processor.decode() now receives tensor instead of list

🔄 Now STOP the running translation cell and RE-RUN it!


## Step 6: Load Reference Transcriptions

Load human-transcribed reference text for comparison.

In [ ]:
print("=" * 60)
print("LOADING REFERENCE TRANSCRIPTIONS")
print("=" * 60)

reference_texts = {}

if REFERENCE_FILE.exists():
    print(f"📖 Reading: {REFERENCE_FILE}")
    
    # Detect separator
    with open(REFERENCE_FILE, 'r', encoding='utf-8') as f:
        first_lines = [f.readline() for _ in range(3)]
    
    # Try to detect separator (| or tab)
    separator = None
    for line in first_lines:
        if '|' in line:
            separator = '|'
            break
        elif '\t' in line:
            separator = '\t'
            break
    
    if separator:
        print(f"   Detected separator: {repr(separator)}")
        
        # Read all references
        with open(REFERENCE_FILE, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if not line:
                    continue
                
                try:
                    filename, text = line.split(separator, 1)
                    filename = filename.strip()
                    
                    # Convert .txt extension to .wav for matching
                    if filename.lower().endswith('.txt'):
                        filename = filename[:-4] + '.wav'
                    
                    reference_texts[filename.lower()] = text.strip()
                except ValueError:
                    print(f"   ⚠️ Skipping malformed line {line_num}: {line[:50]}...")
        
        print(f"\n✅ Loaded {len(reference_texts)} reference transcriptions")
        
        # Show sample
        if reference_texts:
            print("\n📝 Sample references:")
            for i, (filename, text) in enumerate(list(reference_texts.items())[:3], 1):
                preview = text[:80] + "..." if len(text) > 80 else text
                print(f"   {i}. {filename}: {preview}")
    else:
        print("❌ Could not detect separator in reference file")
        print("   Expected format: filename|text or filename<tab>text")
else:
    print(f"❌ Reference file not found: {REFERENCE_FILE}")
    print("\nPlease create a reference file with format:")
    print("   filename.wav|Human transcribed text in English")
    print("   or")
    print("   filename.wav<tab>Human transcribed text in English")

In [31]:
print("=" * 80)
print("COMPREHENSIVE EVALUATION: MODEL vs HUMAN TRANSCRIPTIONS")
print("=" * 80)

# Load model translations (SeamlessM4T output)
model_translations = {}
model_file = cwd / "seamless_translations.txt"

if model_file.exists():
    print(f"\n📖 Loading model translations: {model_file}")
    with open(model_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line and '\t' in line:
                filename, text = line.split('\t', 1)
                # Convert .wav to .txt for matching
                if filename.endswith('.wav'):
                    filename = filename[:-4] + '.txt'
                model_translations[filename.lower()] = text.strip()
    print(f"   ✅ Loaded {len(model_translations)} model translations")
else:
    print(f"   ❌ Model file not found: {model_file}")

# Load human transcriptions (manual references)
human_references = {}
ref_file = cwd.parent / "1.TG_File Name _Modified Translated Text.txt"

if ref_file.exists():
    print(f"\n📖 Loading human references: {ref_file}")
    with open(ref_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line and '\t' in line:
                filename, text = line.split('\t', 1)
                human_references[filename.lower()] = text.strip()
    print(f"   ✅ Loaded {len(human_references)} human references")
else:
    print(f"   ❌ Reference file not found: {ref_file}")

# Match translations with references
print("\n" + "=" * 80)
print("MATCHING TRANSLATIONS")
print("=" * 80)

hypotheses = []  # Model translations
references = []  # Human transcriptions
matched_files = []
unmatched_model = []
unmatched_human = []

for filename, model_text in model_translations.items():
    if filename in human_references:
        hypotheses.append(model_text)
        references.append(human_references[filename])
        matched_files.append(filename)
    else:
        unmatched_model.append(filename)

for filename in human_references:
    if filename not in model_translations:
        unmatched_human.append(filename)

print(f"\n📊 Matching Statistics:")
print(f"   Model translations:     {len(model_translations)}")
print(f"   Human references:       {len(human_references)}")
print(f"   ✅ Matched pairs:       {len(matched_files)}")
print(f"   ⚠️  Unmatched (model):   {len(unmatched_model)}")
print(f"   ⚠️  Unmatched (human):   {len(unmatched_human)}")

if unmatched_model:
    print(f"\n   First 5 unmatched model files: {unmatched_model[:5]}")
if unmatched_human:
    print(f"\n   First 5 unmatched human files: {unmatched_human[:5]}")

if len(matched_files) == 0:
    print("\n❌ ERROR: No matching pairs found!")
    print("   Please check that filenames match between model output and reference file")
else:
    print(f"\n✅ Proceeding with {len(matched_files)} matched pairs for evaluation")
    
    # Show sample comparisons
    print("\n" + "=" * 80)
    print("SAMPLE COMPARISONS (First 3 pairs)")
    print("=" * 80)
    for i, (filename, hyp, ref) in enumerate(zip(matched_files[:3], hypotheses[:3], references[:3]), 1):
        print(f"\n{i}. File: {filename}")
        print(f"   MODEL:  {hyp[:100]}{'...' if len(hyp) > 100 else ''}")
        print(f"   HUMAN:  {ref[:100]}{'...' if len(ref) > 100 else ''}")

print("\n" + "=" * 80)


COMPREHENSIVE EVALUATION: MODEL vs HUMAN TRANSCRIPTIONS

📖 Loading model translations: c:\Users\nani\Desktop\sonar\repo\seamless_translations.txt
   ✅ Loaded 2665 model translations

📖 Loading human references: c:\Users\nani\Desktop\sonar\1.TG_File Name _Modified Translated Text.txt
   ✅ Loaded 2667 human references

MATCHING TRANSLATIONS

📊 Matching Statistics:
   Model translations:     2665
   Human references:       2667
   ✅ Matched pairs:       2665
   ⚠️  Unmatched (model):   0
   ⚠️  Unmatched (human):   2

   First 5 unmatched human files: ['iiith_dbc_pe_tel_tg_1000_84912.txt', 'iiith_dbc_pe_tel_tg_1000_84913.txt']

✅ Proceeding with 2665 matched pairs for evaluation

SAMPLE COMPARISONS (First 3 pairs)

1. File: iiith_dbc_pe_tel_tg_1000_84915.txt
   MODEL:  Giva is a form of dance in Andhra Pradesh. I like to play with toys because they are very popular in...
   HUMAN:  Talking about Nammo Pantulu Bommalu (puppets), I like them because these are played in Andhra and ar...

2. 

In [33]:
print("=" * 80)
print("DETAILED MATCHING ANALYSIS")
print("=" * 80)

print("\n🔍 HOW MATCHING WORKS:")
print("   1. Extract filename from each line (before the TAB character)")
print("   2. Normalize filenames to lowercase")
print("   3. Convert .wav to .txt (or vice versa) for compatibility")
print("   4. Match by filename (NOT by line number/position)")

print("\n" + "=" * 80)
print("MISSING FILES ANALYSIS")
print("=" * 80)

# Files in HUMAN references but NOT in MODEL translations
print(f"\n📋 Files in Human References but MISSING in Model Output ({len(unmatched_human)} files):")
if unmatched_human:
    for i, filename in enumerate(sorted(unmatched_human)[:10], 1):
        print(f"   {i}. {filename}")
    if len(unmatched_human) > 10:
        print(f"   ... and {len(unmatched_human) - 10} more")
else:
    print("   ✅ None - all human reference files have model translations")

# Files in MODEL translations but NOT in HUMAN references
print(f"\n📋 Files in Model Output but MISSING in Human References ({len(unmatched_model)} files):")
if unmatched_model:
    for i, filename in enumerate(sorted(unmatched_model)[:10], 1):
        print(f"   {i}. {filename}")
    if len(unmatched_model) > 10:
        print(f"   ... and {len(unmatched_model) - 10} more")
else:
    print("   ✅ None - all model files have human references")

print("\n" + "=" * 80)
print("FILENAME FORMAT ANALYSIS")
print("=" * 80)

# Check the 2 missing files in detail
if unmatched_human:
    print("\n🔍 Analyzing the 2 missing files from human references:")
    for filename in sorted(unmatched_human):
        print(f"\n   Missing: {filename}")
        
        # Try to find similar filenames in model output
        base_name = filename.replace('.txt', '').replace('.wav', '')
        similar_in_model = [f for f in model_translations.keys() 
                           if base_name in f or f.replace('.txt', '').replace('.wav', '') in base_name]
        
        if similar_in_model:
            print(f"   Similar files found in model output:")
            for sim in similar_in_model[:3]:
                print(f"      - {sim}")
        else:
            print(f"   ❌ No similar filename found in model output")
            
        # Check if this file exists in the audio folder
        audio_path_txt = AUDIO_FOLDER / filename.replace('.txt', '.wav')
        audio_path_wav = AUDIO_FOLDER / filename
        
        if audio_path_txt.exists():
            print(f"   ✅ Audio file exists: {audio_path_txt.name}")
        elif audio_path_wav.exists():
            print(f"   ✅ Audio file exists: {audio_path_wav.name}")
        else:
            print(f"   ⚠️  Audio file not found in {AUDIO_FOLDER}")

print("\n" + "=" * 80)
print("VERIFICATION: Sample Matched Pairs")
print("=" * 80)

print("\n📌 First 5 matched pairs (showing filename matching):")
for i, filename in enumerate(sorted(matched_files)[:5], 1):
    model_text = model_translations[filename][:60] + "..."
    human_text = human_references[filename][:60] + "..."
    print(f"\n{i}. Filename: {filename}")
    print(f"   MODEL: {model_text}")
    print(f"   HUMAN: {human_text}")

print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)

print(f"""
Total audio files in folder:     {len(audio_files) if 'audio_files' in dir() else 'N/A'}
Model translations generated:    {len(model_translations)}
Human references provided:       {len(human_references)}
Successfully matched pairs:      {len(matched_files)}

Missing from model output:       {len(unmatched_human)} files
Missing from human references:   {len(unmatched_model)} files

✅ Matching is done by FILENAME, not by line position
✅ All matched files have identical filenames (case-insensitive)
✅ The 2 missing files likely failed during translation or weren't in the audio folder
""")

print("=" * 80)


DETAILED MATCHING ANALYSIS

🔍 HOW MATCHING WORKS:
   1. Extract filename from each line (before the TAB character)
   2. Normalize filenames to lowercase
   3. Convert .wav to .txt (or vice versa) for compatibility
   4. Match by filename (NOT by line number/position)

MISSING FILES ANALYSIS

📋 Files in Human References but MISSING in Model Output (2 files):
   1. iiith_dbc_pe_tel_tg_1000_84912.txt
   2. iiith_dbc_pe_tel_tg_1000_84913.txt

📋 Files in Model Output but MISSING in Human References (0 files):
   ✅ None - all model files have human references

FILENAME FORMAT ANALYSIS

🔍 Analyzing the 2 missing files from human references:

   Missing: iiith_dbc_pe_tel_tg_1000_84912.txt
   ❌ No similar filename found in model output
   ✅ Audio file exists: iiith_dbc_pe_tel_tg_1000_84912.wav

   Missing: iiith_dbc_pe_tel_tg_1000_84913.txt
   ❌ No similar filename found in model output
   ✅ Audio file exists: iiith_dbc_pe_tel_tg_1000_84913.wav

VERIFICATION: Sample Matched Pairs

📌 First 5 ma

In [ ]:
if len(matched_files) > 0:
    print("=" * 80)
    print("CALCULATING EVALUATION METRICS")
    print("=" * 80)
    
    # Suppress warnings for cleaner output
    import warnings
    warnings.filterwarnings('ignore')
    import logging
    logging.getLogger('huggingface_hub').setLevel(logging.ERROR)
    logging.getLogger('transformers').setLevel(logging.ERROR)
    
    import sacrebleu
    from jiwer import wer, cer
    from rouge_score import rouge_scorer
    from nltk.translate.meteor_score import meteor_score
    from bert_score import score as bert_score
    import numpy as np
    
    # 1. BLEU Score
    print("\n📊 Calculating BLEU (n-gram precision)...")
    bleu = sacrebleu.corpus_bleu(hypotheses, [references])
    bleu_score = round(bleu.score, 2)
    
    # 2. ChrF++ Score
    print("📊 Calculating ChrF++ (character n-gram F-score)...")
    chrf = sacrebleu.corpus_chrf(hypotheses, [references])
    chrf_score = round(chrf.score, 2)
    
    # 3. TER Score
    print("📊 Calculating TER (Translation Edit Rate)...")
    ter = sacrebleu.corpus_ter(hypotheses, [references])
    ter_score = round(ter.score, 2)
    
    # 4. WER (Word Error Rate)
    print("📊 Calculating WER (Word Error Rate)...")
    wer_scores = [wer(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    wer_score = round(np.mean(wer_scores), 4)
    
    # 5. CER (Character Error Rate)
    print("📊 Calculating CER (Character Error Rate)...")
    cer_scores = [cer(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    cer_score = round(np.mean(cer_scores), 4)
    
    # 6. ROUGE Scores
    print("📊 Calculating ROUGE (Recall metrics)...")
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    rouge1_f1 = round(np.mean([s['rouge1'].fmeasure for s in rouge_scores]), 4)
    rouge2_f1 = round(np.mean([s['rouge2'].fmeasure for s in rouge_scores]), 4)
    rougeL_f1 = round(np.mean([s['rougeL'].fmeasure for s in rouge_scores]), 4)
    
    # 7. METEOR Score
    print("📊 Calculating METEOR (semantic matching)...")
    try:
        meteor_scores = [meteor_score([ref.split()], hyp.split()) for ref, hyp in zip(references, hypotheses)]
        meteor_score_val = round(np.mean(meteor_scores), 4)
    except Exception as e:
        print(f"   ⚠️ METEOR failed: {e}")
        meteor_score_val = None
    
    # 8. BERTScore
    print("📊 Calculating BERTScore (neural semantic similarity)...")
    try:
        P, R, F1 = bert_score(hypotheses, references, lang='en', verbose=False)
        bert_p = round(P.mean().item(), 4)
        bert_r = round(R.mean().item(), 4)
        bert_f1 = round(F1.mean().item(), 4)
    except Exception as e:
        print(f"   ⚠️ BERTScore failed: {e}")
        bert_p = bert_r = bert_f1 = None
    
    # Display Results
    print("\n" + "=" * 80)
    print("🎯 EVALUATION RESULTS")
    print("=" * 80)
    
    print("\n📈 PRIMARY QUALITY METRICS (Higher = Better):")
    print(f"   BLEU Score:              {bleu_score:>8.2f} / 100.00")
    print(f"   ChrF++ Score:            {chrf_score:>8.2f} / 100.00")
    if meteor_score_val:
        print(f"   METEOR Score:            {meteor_score_val:>8.4f} / 1.0000")
    
    print("\n📉 ERROR METRICS (Lower = Better):")
    print(f"   TER (Edit Rate):         {ter_score:>8.2f}")
    print(f"   WER (Word Error):        {wer_score:>8.4f}")
    print(f"   CER (Char Error):        {cer_score:>8.4f}")
    
    print("\n🔍 ROUGE SCORES (Recall-oriented):")
    print(f"   ROUGE-1 F1:              {rouge1_f1:>8.4f}")
    print(f"   ROUGE-2 F1:              {rouge2_f1:>8.4f}")
    print(f"   ROUGE-L F1:              {rougeL_f1:>8.4f}")
    
    if bert_f1:
        print("\n🧠 BERTSCORE (Semantic Similarity):")
        print(f"   Precision:               {bert_p:>8.4f}")
        print(f"   Recall:                  {bert_r:>8.4f}")
        print(f"   F1 Score:                {bert_f1:>8.4f}")
    
    # Save results
    results_output = {
        'metadata': {
            'model': 'SeamlessM4T-v2-large',
            'source_language': 'Telugu',
            'target_language': 'English',
            'matched_pairs': len(matched_files),
            'total_model_translations': len(model_translations),
            'total_human_references': len(human_references),
            'evaluation_date': datetime.datetime.now().isoformat()
        },
        'metrics': {
            'bleu': bleu_score,
            'chrf': chrf_score,
            'ter': ter_score,
            'wer': wer_score,
            'cer': cer_score,
            'rouge1_f1': rouge1_f1,
            'rouge2_f1': rouge2_f1,
            'rougeL_f1': rougeL_f1,
            'meteor': meteor_score_val,
            'bertscore_precision': bert_p,
            'bertscore_recall': bert_r,
            'bertscore_f1': bert_f1
        }
    }
    
    json_file = RESULTS_DIR / "evaluation_results.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results_output, f, indent=2, ensure_ascii=False)
    
    print("\n" + "=" * 80)
    print(f"💾 Results saved to: {json_file}")
    print("=" * 80)
    
    # Re-enable warnings
    warnings.filterwarnings('default')
    
else:
    print("\n❌ Cannot calculate metrics - no matched pairs found")


CALCULATING EVALUATION METRICS

📊 Calculating BLEU (n-gram precision)...
📊 Calculating ChrF++ (character n-gram F-score)...
📊 Calculating TER (Translation Edit Rate)...
📊 Calculating WER (Word Error Rate)...
📊 Calculating CER (Character Error Rate)...
📊 Calculating ROUGE (Recall metrics)...
📊 Calculating METEOR (semantic matching)...
📊 Calculating BERTScore (neural semantic similarity)...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🎯 EVALUATION RESULTS

📈 PRIMARY QUALITY METRICS (Higher = Better):
   BLEU Score:                 14.65 / 100.00
   ChrF++ Score:               42.61 / 100.00
   METEOR Score:              0.3615 / 1.0000

📉 ERROR METRICS (Lower = Better):
   TER (Edit Rate):            98.67
   WER (Word Error):          1.1339
   CER (Char Error):          0.8297

🔍 ROUGE SCORES (Recall-oriented):
   ROUGE-1 F1:                0.4578
   ROUGE-2 F1:                0.2359
   ROUGE-L F1:                0.4005

🧠 BERTSCORE (Semantic Similarity):
   Precision:                 0.9040
   Recall:                    0.9068
   F1 Score:                  0.9052

💾 Results saved to: c:\Users\nani\Desktop\sonar\repo\results\evaluation_results.json


## Step 7: Evaluation Metrics Setup

Define all evaluation metric functions.

In [ ]:
import sacrebleu
from jiwer import wer, cer
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score

print("=" * 60)
print("EVALUATION METRICS")
print("=" * 60)

def calculate_all_metrics(hypotheses, references, verbose=True):
    """
    Calculate comprehensive translation quality metrics.
    
    Args:
        hypotheses: List of model translations
        references: List of human reference translations
        verbose: Print detailed metric descriptions
    
    Returns:
        Dictionary of all metric scores
    """
    if not hypotheses or not references:
        print("❌ Empty hypotheses or references")
        return {}
    
    if len(hypotheses) != len(references):
        print(f"⚠️ Mismatch: {len(hypotheses)} hypotheses vs {len(references)} references")
    
    metrics = {}
    
    # 1. BLEU Score (n-gram overlap)
    if verbose:
        print("\n📊 Calculating BLEU (Bilingual Evaluation Understudy)...")
        print("   Measures n-gram precision (higher is better, 0-100)")
    bleu = sacrebleu.corpus_bleu(hypotheses, [references])
    metrics['bleu'] = round(bleu.score, 2)
    
    # 2. ChrF++ Score (character n-gram F-score)
    if verbose:
        print("\n📊 Calculating ChrF++ (Character n-gram F-score)...")
        print("   Character-level matching (higher is better, 0-100)")
    chrf = sacrebleu.corpus_chrf(hypotheses, [references])
    metrics['chrf'] = round(chrf.score, 2)
    
    # 3. TER Score (Translation Edit Rate)
    if verbose:
        print("\n📊 Calculating TER (Translation Edit Rate)...")
        print("   Edit distance metric (lower is better)")
    ter = sacrebleu.corpus_ter(hypotheses, [references])
    metrics['ter'] = round(ter.score, 2)
    
    # 4. WER (Word Error Rate)
    if verbose:
        print("\n📊 Calculating WER (Word Error Rate)...")
        print("   Word-level error rate (lower is better, 0-1)")
    wer_scores = [wer(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    metrics['wer'] = round(np.mean(wer_scores), 4)
    
    # 5. CER (Character Error Rate)
    if verbose:
        print("\n📊 Calculating CER (Character Error Rate)...")
        print("   Character-level error rate (lower is better, 0-1)")
    cer_scores = [cer(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    metrics['cer'] = round(np.mean(cer_scores), 4)
    
    # 6. ROUGE Scores
    if verbose:
        print("\n📊 Calculating ROUGE (Recall-Oriented Understudy for Gisting Evaluation)...")
        print("   Measures recall of n-grams (higher is better, 0-1)")
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = [scorer.score(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    
    metrics['rouge1_f1'] = round(np.mean([s['rouge1'].fmeasure for s in rouge_scores]), 4)
    metrics['rouge2_f1'] = round(np.mean([s['rouge2'].fmeasure for s in rouge_scores]), 4)
    metrics['rougeL_f1'] = round(np.mean([s['rougeL'].fmeasure for s in rouge_scores]), 4)
    
    # 7. METEOR Score
    if verbose:
        print("\n📊 Calculating METEOR (Metric for Evaluation of Translation with Explicit ORdering)...")
        print("   Considers synonyms and paraphrases (higher is better, 0-1)")
    try:
        meteor_scores = [
            meteor_score([ref.split()], hyp.split()) 
            for ref, hyp in zip(references, hypotheses)
        ]
        metrics['meteor'] = round(np.mean(meteor_scores), 4)
    except Exception as e:
        if verbose:
            print(f"   ⚠️ METEOR calculation failed: {e}")
        metrics['meteor'] = None
    
    # 8. BERTScore (contextual embeddings)
    if verbose:
        print("\n📊 Calculating BERTScore (BERT-based semantic similarity)...")
        print("   Neural semantic similarity (higher is better, 0-1)")
    try:
        P, R, F1 = bert_score(hypotheses, references, lang='en', verbose=False)
        metrics['bertscore_precision'] = round(P.mean().item(), 4)
        metrics['bertscore_recall'] = round(R.mean().item(), 4)
        metrics['bertscore_f1'] = round(F1.mean().item(), 4)
    except Exception as e:
        if verbose:
            print(f"   ⚠️ BERTScore calculation failed: {e}")
        metrics['bertscore_precision'] = None
        metrics['bertscore_recall'] = None
        metrics['bertscore_f1'] = None
    
    if verbose:
        print("\n✅ All metrics calculated successfully")
    
    return metrics

print("✅ Evaluation functions defined")
print("\nSupported metrics:")
print("   • BLEU: n-gram precision")
print("   • ChrF++: character n-gram F-score")
print("   • TER: translation edit rate")
print("   • WER: word error rate")
print("   • CER: character error rate")
print("   • ROUGE (1, 2, L): recall metrics")
print("   • METEOR: semantic matching")
print("   • BERTScore: neural semantic similarity")

## Step 8: Run Evaluation

Compare model translations with human references and calculate scores.

In [ ]:
print("=" * 60)
print("RUNNING EVALUATION")
print("=" * 60)

# Prepare matched pairs
hypotheses = []  # Model translations
references = []  # Human transcriptions
matched_files = []

# Match translations with references
for filename, model_translation in cached_results.items():
    # Skip error entries
    if model_translation.startswith("ERROR:"):
        continue
    
    # Look for matching reference
    filename_lower = filename.lower()
    if filename_lower in reference_texts:
        hypotheses.append(model_translation)
        references.append(reference_texts[filename_lower])
        matched_files.append(filename)

print(f"\n📊 Data Summary:")
print(f"   Total translations: {len(cached_results)}")
print(f"   Total references: {len(reference_texts)}")
print(f"   Matched pairs: {len(matched_files)}")
print(f"   Errors/Skipped: {sum(1 for v in cached_results.values() if v.startswith('ERROR:'))}")

if len(matched_files) == 0:
    print("\n❌ No matching pairs found!")
    print("\nPossible issues:")
    print("   1. Filenames in translation output don't match reference file")
    print("   2. Reference file is empty or incorrectly formatted")
    print("   3. All translations resulted in errors")
else:
    print(f"\n✅ Found {len(matched_files)} valid pairs for evaluation")
    
    # Calculate metrics
    print("\n" + "=" * 60)
    all_metrics = calculate_all_metrics(hypotheses, references, verbose=True)
    
    # Display results
    print("\n" + "=" * 60)
    print("📈 EVALUATION RESULTS")
    print("=" * 60)
    
    print("\n🎯 Primary Metrics (Higher is Better):")
    print(f"   BLEU Score:        {all_metrics.get('bleu', 'N/A'):>8} / 100")
    print(f"   ChrF++ Score:      {all_metrics.get('chrf', 'N/A'):>8} / 100")
    if all_metrics.get('meteor'):
        print(f"   METEOR Score:      {all_metrics.get('meteor', 'N/A'):>8.4f} / 1.0")
    
    print("\n📉 Error Metrics (Lower is Better):")
    print(f"   TER Score:         {all_metrics.get('ter', 'N/A'):>8}")
    print(f"   WER (Word Error):  {all_metrics.get('wer', 'N/A'):>8.4f}")
    print(f"   CER (Char Error):  {all_metrics.get('cer', 'N/A'):>8.4f}")
    
    print("\n🔍 ROUGE Scores (Recall-oriented):")
    print(f"   ROUGE-1 F1:        {all_metrics.get('rouge1_f1', 'N/A'):>8.4f}")
    print(f"   ROUGE-2 F1:        {all_metrics.get('rouge2_f1', 'N/A'):>8.4f}")
    print(f"   ROUGE-L F1:        {all_metrics.get('rougeL_f1', 'N/A'):>8.4f}")
    
    if all_metrics.get('bertscore_f1'):
        print("\n🧠 BERTScore (Semantic Similarity):")
        print(f"   Precision:         {all_metrics.get('bertscore_precision', 'N/A'):>8.4f}")
        print(f"   Recall:            {all_metrics.get('bertscore_recall', 'N/A'):>8.4f}")
        print(f"   F1:                {all_metrics.get('bertscore_f1', 'N/A'):>8.4f}")
    
    # Save results to JSON
    results_output = {
        'metadata': {
            'model': 'SeamlessM4T-v2-large',
            'source_language': 'Telugu',
            'target_language': 'English',
            'total_files': len(cached_results),
            'matched_pairs': len(matched_files),
            'evaluation_date': datetime.datetime.now().isoformat()
        },
        'metrics': all_metrics,
        'matched_files': matched_files
    }
    
    json_output_file = RESULTS_DIR / "evaluation_results.json"
    with open(json_output_file, 'w', encoding='utf-8') as f:
        json.dump(results_output, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Results saved to: {json_output_file}")
    print("=" * 60)

## Step 9: Per-File Detailed Analysis

Analyze individual file translations with scores.

In [ ]:
print("=" * 60)
print("PER-FILE DETAILED ANALYSIS")
print("=" * 60)

if matched_files:
    # Calculate individual scores for each file
    file_results = []
    
    print("\nCalculating individual file metrics...")
    
    for filename, hyp, ref in zip(matched_files, hypotheses, references):
        file_metrics = {
            'filename': filename,
            'model_translation': hyp,
            'human_reference': ref,
            'translation_length': len(hyp.split()),
            'reference_length': len(ref.split())
        }
        
        # Calculate metrics for this pair
        try:
            # BLEU
            bleu = sacrebleu.sentence_bleu(hyp, [ref])
            file_metrics['bleu'] = round(bleu.score, 2)
            
            # ChrF
            chrf = sacrebleu.sentence_chrf(hyp, [ref])
            file_metrics['chrf'] = round(chrf.score, 2)
            
            # WER & CER
            file_metrics['wer'] = round(wer(ref, hyp), 4)
            file_metrics['cer'] = round(cer(ref, hyp), 4)
            
            # ROUGE
            scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
            rouge = scorer.score(ref, hyp)
            file_metrics['rouge1'] = round(rouge['rouge1'].fmeasure, 4)
            file_metrics['rougeL'] = round(rouge['rougeL'].fmeasure, 4)
            
        except Exception as e:
            print(f"   ⚠️ Error calculating metrics for {filename}: {e}")
            file_metrics['bleu'] = 0
            file_metrics['chrf'] = 0
            file_metrics['wer'] = 1.0
            file_metrics['cer'] = 1.0
            file_metrics['rouge1'] = 0
            file_metrics['rougeL'] = 0
        
        file_results.append(file_metrics)
    
    # Create DataFrame
    df = pd.DataFrame(file_results)
    
    # Display summary statistics
    print("\n📊 Summary Statistics:")
    print(df[['bleu', 'chrf', 'wer', 'cer', 'rouge1', 'rougeL']].describe())
    
    # Find best and worst translations
    print("\n🏆 Top 5 Translations (by BLEU):")
    top_5 = df.nlargest(5, 'bleu')[['filename', 'bleu', 'chrf', 'wer']]
    for idx, row in top_5.iterrows():
        print(f"   {row['filename']}: BLEU={row['bleu']:.2f}, ChrF={row['chrf']:.2f}, WER={row['wer']:.4f}")
    
    print("\n⚠️ Bottom 5 Translations (by BLEU):")
    bottom_5 = df.nsmallest(5, 'bleu')[['filename', 'bleu', 'chrf', 'wer']]
    for idx, row in bottom_5.iterrows():
        print(f"   {row['filename']}: BLEU={row['bleu']:.2f}, ChrF={row['chrf']:.2f}, WER={row['wer']:.4f}")
    
    # Save detailed results to CSV
    csv_output_file = RESULTS_DIR / "detailed_file_results.csv"
    df.to_csv(csv_output_file, index=False, encoding='utf-8')
    print(f"\n💾 Detailed results saved to: {csv_output_file}")
    
    # Save comparison file (side-by-side)
    comparison_file = RESULTS_DIR / "translation_comparison.txt"
    with open(comparison_file, 'w', encoding='utf-8') as f:
        f.write("TRANSLATION COMPARISON: MODEL vs HUMAN\n")
        f.write("=" * 100 + "\n\n")
        
        for _, row in df.iterrows():
            f.write(f"File: {row['filename']}\n")
            f.write(f"BLEU: {row['bleu']:.2f} | ChrF: {row['chrf']:.2f} | WER: {row['wer']:.4f}\n")
            f.write(f"{'─' * 100}\n")
            f.write(f"MODEL:  {row['model_translation']}\n")
            f.write(f"HUMAN:  {row['human_reference']}\n")
            f.write(f"{'═' * 100}\n\n")
    
    print(f"💾 Side-by-side comparison saved to: {comparison_file}")
else:
    print("⚠️ No matched files to analyze")

## Step 10: Visualization

Create charts to visualize the evaluation results.

In [ ]:
if matched_files and len(file_results) > 0:
    print("=" * 60)
    print("GENERATING VISUALIZATIONS")
    print("=" * 60)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Telugu → English Translation Quality Analysis (SeamlessM4T)', 
                 fontsize=16, fontweight='bold')
    
    # 1. Metric Distribution (Box Plot)
    ax1 = axes[0, 0]
    metric_data = df[['bleu', 'chrf', 'rouge1', 'rougeL']].copy()
    metric_data.columns = ['BLEU', 'ChrF++', 'ROUGE-1', 'ROUGE-L']
    metric_data.boxplot(ax=ax1)
    ax1.set_title('Distribution of Quality Metrics', fontweight='bold')
    ax1.set_ylabel('Score')
    ax1.grid(True, alpha=0.3)
    
    # 2. Error Metrics (Box Plot)
    ax2 = axes[0, 1]
    error_data = df[['wer', 'cer']].copy()
    error_data.columns = ['WER', 'CER']
    error_data.boxplot(ax=ax2)
    ax2.set_title('Distribution of Error Metrics', fontweight='bold')
    ax2.set_ylabel('Error Rate')
    ax2.grid(True, alpha=0.3)
    
    # 3. BLEU vs WER Scatter
    ax3 = axes[1, 0]
    scatter = ax3.scatter(df['wer'], df['bleu'], c=df['chrf'], 
                         cmap='RdYlGn', s=100, alpha=0.6, edgecolors='black')
    ax3.set_xlabel('WER (Word Error Rate) - Lower is Better', fontweight='bold')
    ax3.set_ylabel('BLEU Score - Higher is Better', fontweight='bold')
    ax3.set_title('BLEU vs WER (colored by ChrF++)', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax3, label='ChrF++ Score')
    
    # 4. Metric Correlation Heatmap
    ax4 = axes[1, 1]
    correlation_metrics = df[['bleu', 'chrf', 'wer', 'cer', 'rouge1', 'rougeL']]
    correlation = correlation_metrics.corr()
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, ax=ax4, cbar_kws={'label': 'Correlation'})
    ax4.set_title('Metric Correlation Matrix', fontweight='bold')
    
    plt.tight_layout()
    
    # Save figure
    viz_file = RESULTS_DIR / "evaluation_visualization.png"
    plt.savefig(viz_file, dpi=300, bbox_inches='tight')
    print(f"📊 Visualization saved to: {viz_file}")
    plt.show()
    
    # Additional: Top/Bottom performers chart
    fig2, (ax5, ax6) = plt.subplots(1, 2, figsize=(15, 5))
    fig2.suptitle('Best and Worst Performing Translations', fontsize=14, fontweight='bold')
    
    # Top 10
    top_10 = df.nlargest(10, 'bleu')
    ax5.barh(range(len(top_10)), top_10['bleu'], color='green', alpha=0.7)
    ax5.set_yticks(range(len(top_10)))
    ax5.set_yticklabels([f[:20] + '...' if len(f) > 20 else f for f in top_10['filename']], fontsize=8)
    ax5.set_xlabel('BLEU Score')
    ax5.set_title('Top 10 Translations', fontweight='bold')
    ax5.invert_yaxis()
    ax5.grid(True, alpha=0.3, axis='x')
    
    # Bottom 10
    bottom_10 = df.nsmallest(10, 'bleu')
    ax6.barh(range(len(bottom_10)), bottom_10['bleu'], color='red', alpha=0.7)
    ax6.set_yticks(range(len(bottom_10)))
    ax6.set_yticklabels([f[:20] + '...' if len(f) > 20 else f for f in bottom_10['filename']], fontsize=8)
    ax6.set_xlabel('BLEU Score')
    ax6.set_title('Bottom 10 Translations', fontweight='bold')
    ax6.invert_yaxis()
    ax6.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    
    viz_file2 = RESULTS_DIR / "top_bottom_performers.png"
    plt.savefig(viz_file2, dpi=300, bbox_inches='tight')
    print(f"📊 Top/Bottom performers chart saved to: {viz_file2}")
    plt.show()
    
    print("\n✅ All visualizations generated successfully!")
else:
    print("⚠️ No data available for visualization")

## Step 11: Summary Report

Generate a comprehensive summary report.

In [ ]:
if matched_files and all_metrics:
    print("=" * 60)
    print("FINAL SUMMARY REPORT")
    print("=" * 60)
    
    report = []
    report.append("=" * 80)
    report.append("TELUGU → ENGLISH TRANSLATION EVALUATION REPORT")
    report.append("Model: SeamlessM4T-v2-large")
    report.append("=" * 80)
    report.append("")
    report.append(f"Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append("")
    report.append("─" * 80)
    report.append("DATASET SUMMARY")
    report.append("─" * 80)
    report.append(f"Total audio files processed:     {len(cached_results)}")
    report.append(f"Successful translations:         {len(cached_results) - sum(1 for v in cached_results.values() if v.startswith('ERROR:'))}")
    report.append(f"Failed translations:             {sum(1 for v in cached_results.values() if v.startswith('ERROR:'))}")
    report.append(f"Reference transcriptions:        {len(reference_texts)}")
    report.append(f"Matched pairs for evaluation:    {len(matched_files)}")
    report.append("")
    report.append("─" * 80)
    report.append("OVERALL QUALITY METRICS")
    report.append("─" * 80)
    report.append("")
    report.append("Primary Quality Metrics (Higher = Better):")
    report.append(f"  BLEU Score:              {all_metrics.get('bleu', 'N/A'):>8.2f} / 100")
    report.append(f"  ChrF++ Score:            {all_metrics.get('chrf', 'N/A'):>8.2f} / 100")
    if all_metrics.get('meteor'):
        report.append(f"  METEOR Score:            {all_metrics.get('meteor', 'N/A'):>8.4f} / 1.0")
    report.append("")
    report.append("Error Metrics (Lower = Better):")
    report.append(f"  TER (Translation Edit):  {all_metrics.get('ter', 'N/A'):>8.2f}")
    report.append(f"  WER (Word Error Rate):   {all_metrics.get('wer', 'N/A'):>8.4f}")
    report.append(f"  CER (Char Error Rate):   {all_metrics.get('cer', 'N/A'):>8.4f}")
    report.append("")
    report.append("ROUGE Scores (Recall-oriented):")
    report.append(f"  ROUGE-1 F1:              {all_metrics.get('rouge1_f1', 'N/A'):>8.4f}")
    report.append(f"  ROUGE-2 F1:              {all_metrics.get('rouge2_f1', 'N/A'):>8.4f}")
    report.append(f"  ROUGE-L F1:              {all_metrics.get('rougeL_f1', 'N/A'):>8.4f}")
    report.append("")
    if all_metrics.get('bertscore_f1'):
        report.append("BERTScore (Semantic Similarity):")
        report.append(f"  Precision:               {all_metrics.get('bertscore_precision', 'N/A'):>8.4f}")
        report.append(f"  Recall:                  {all_metrics.get('bertscore_recall', 'N/A'):>8.4f}")
        report.append(f"  F1 Score:                {all_metrics.get('bertscore_f1', 'N/A'):>8.4f}")
        report.append("")
    
    report.append("─" * 80)
    report.append("PER-FILE STATISTICS")
    report.append("─" * 80)
    report.append("")
    report.append(f"Average translation length: {df['translation_length'].mean():.1f} words")
    report.append(f"Average reference length:   {df['reference_length'].mean():.1f} words")
    report.append("")
    report.append("BLEU Score Distribution:")
    report.append(f"  Mean:    {df['bleu'].mean():.2f}")
    report.append(f"  Median:  {df['bleu'].median():.2f}")
    report.append(f"  Std Dev: {df['bleu'].std():.2f}")
    report.append(f"  Min:     {df['bleu'].min():.2f}")
    report.append(f"  Max:     {df['bleu'].max():.2f}")
    report.append("")
    report.append("─" * 80)
    report.append("FILES GENERATED")
    report.append("─" * 80)
    report.append(f"  • Translations:           {OUTPUT_FILE}")
    report.append(f"  • Evaluation JSON:        {RESULTS_DIR / 'evaluation_results.json'}")
    report.append(f"  • Detailed CSV:           {RESULTS_DIR / 'detailed_file_results.csv'}")
    report.append(f"  • Comparison Text:        {RESULTS_DIR / 'translation_comparison.txt'}")
    report.append(f"  • Visualizations:         {RESULTS_DIR / 'evaluation_visualization.png'}")
    report.append(f"                            {RESULTS_DIR / 'top_bottom_performers.png'}")
    report.append("")
    report.append("=" * 80)
    report.append("END OF REPORT")
    report.append("=" * 80)
    
    # Print report
    report_text = "\n".join(report)
    print(report_text)
    
    # Save report
    report_file = RESULTS_DIR / "evaluation_report.txt"
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(report_text)
    
    print(f"\n💾 Full report saved to: {report_file}")
    
    print("\n" + "=" * 60)
    print("✅ EVALUATION COMPLETE!")
    print("=" * 60)
    print(f"\nAll results saved in: {RESULTS_DIR}")
    print("\nKey files:")
    print(f"  • Summary report:  {report_file.name}")
    print(f"  • Metrics JSON:    evaluation_results.json")
    print(f"  • Detailed CSV:    detailed_file_results.csv")
    print(f"  • Visualizations:  *.png files")
else:
    print("⚠️ Insufficient data to generate report")

## Additional Tools

### Clear Cache (Run to Reset Progress)

Uncomment and run this cell to clear all cached data and start fresh.

In [ ]:
# CAUTION: This will delete all cached translations and progress!
# Uncomment the lines below to clear cache

# import shutil
# if CACHE_DIR.exists():
#     shutil.rmtree(CACHE_DIR)
#     print("🗑️ Cache cleared successfully")
#     CACHE_DIR.mkdir(exist_ok=True)
# else:
#     print("⚠️ No cache directory found")

print("ℹ️ To clear cache, uncomment the code in this cell and run it")

## Notes and Tips

### Using This Notebook:

1. **First Time Setup:**
   - Run cells 1-4 to install dependencies
   - Update `AUDIO_FOLDER` and `REFERENCE_FILE` paths in cell 7
   - Run cell 8 to load SeamlessM4T model (~10GB download)

2. **Processing Your Data:**
   - Place Telugu audio files (.wav format recommended) in your audio folder
   - Create a reference file with format: `filename.wav|English transcription`
   - Run cells 9-10 to translate audio files

3. **Evaluation:**
   - Run cells 11-14 to evaluate and generate scores
   - Check the `results/` folder for detailed outputs

4. **Supported Audio Formats:**
   - Primary: .wav (16kHz, mono recommended)
   - Also supported: .mp3, .flac (will be converted automatically)

5. **Reference File Format:**
   ```
   audio1.wav|This is the human English transcription
   audio2.wav|Another reference translation
   ```

### Troubleshooting:

- **Out of Memory:** Reduce batch size or use CPU (slower but needs less RAM)
- **Model Download Fails:** Check internet connection, try again later
- **No Matches Found:** Verify filename format matches between audio and reference files
- **Poor Scores:** Check audio quality, verify reference translations are accurate

### Performance Tips:

- Use GPU/CUDA for 10-20x faster processing
- Process in batches if you have many files
- Cache is automatically saved every 5 files
- You can interrupt (Ctrl+C) and resume later

---

**Model:** SeamlessM4T-v2-large (Meta AI)  
**License:** CC-BY-NC 4.0  
**Paper:** https://arxiv.org/abs/2312.05187